In [8]:
import os

data_dir = r'E:\users\minghaoli\ICDAR2013 Table COCO\data\icdar2013-competition-dataset-with-gt'
img_dir = r'E:\users\minghaoli\ICDAR2013 Table COCO\data\ICDAR2013_table_coco\images'

In [9]:
from pdf2image import convert_from_path
from tqdm.notebook import tqdm
for pdf_file in tqdm(os.listdir(data_dir)):
    if not pdf_file.endswith('.pdf'):
        continue
    
    imgs = convert_from_path(os.path.join(data_dir, pdf_file))
    basename = pdf_file.replace('.pdf', '')
    for page_i, img in enumerate(imgs):
        img.save(os.path.join(img_dir, '{}_{}_test.jpg'.format(basename, str(page_i))))
        

In [10]:
from PIL import Image
rate_map = {}
for pdf_file in tqdm(os.listdir(data_dir)):
    if not pdf_file.endswith('.pdf'):
        continue
    
    imgs = convert_from_path(os.path.join(data_dir, pdf_file), dpi=72)
    basename = pdf_file.replace('.pdf', '')
    for page_i, img in enumerate(imgs):
        big_img = Image.open(os.path.join(img_dir, '{}_{}_test.jpg'.format(basename, str(page_i))))
        small_img = img
        
        rate_map['{}_{}_test.jpg'.format(basename, str(page_i))] = ((big_img.size[0] / small_img.size[0] + big_img.size[1] / small_img.size[1]) / 2, small_img.size, big_img.size)

In [11]:
import xml.etree.ElementTree as ET
import numpy as np

coco_images = []
coco_annotations = []
coco_categories = [{'id':1, 'name':'table', 'supercategory':'table'}]

img2id = {}

r_image_id = 0
r_annotation_id = 0

for pdf_file in tqdm(os.listdir(data_dir)):
    if not pdf_file.endswith('.pdf'):
        continue
        
    basename = pdf_file.replace('.pdf', '')
    ori_ann_file = basename + '-reg.xml'
    
    tree = ET.parse(open(os.path.join(data_dir, ori_ann_file)))
    root = tree.getroot()
    
    for table in root.findall('table'):
        for region in table.findall('region'):
            page = int(region.attrib['page']) - 1
            img_file = '{}_{}_test.jpg'.format(basename, str(page))
            
            bbox = region.findall('bounding-box')
            assert len(bbox) == 1
            bbox = bbox[0].attrib
            bbox = {k:int(v) for k, v in bbox.items()}
            
            rate = rate_map[img_file][0]
            small_size = rate_map[img_file][1]
            img_size = rate_map[img_file][2]
            bbox = {'xmin':bbox['x1'], 'ymin':small_size[1] - bbox['y2'], 'width':bbox['x2']-bbox['x1'], 'height':bbox['y2']-bbox['y1']}
            bbox = {k:v*rate for k, v in bbox.items()}
            
            if img_file in img2id:
                img_id = img2id[img_file]
            else:
                r_image_id += 1
                img2id[img_file] = r_image_id
                img_id = r_image_id
                coco_images.append({
                    'id':r_image_id,
                    'width':img_size[0],
                    'height':img_size[1],
                    'file_name':img_file                                        
                })
            
            r_annotation_id += 1
            coco_annotations.append({
                'id':r_annotation_id,
                'image_id':img_id,
                'category_id':1,
                'segmentation':[],
                'area':int(bbox['width'] * bbox['height']),
                'bbox':[int(v) for v in [bbox['xmin'], bbox['ymin'], bbox['width'], bbox['height']]]
            })


In [13]:
for img_file in os.listdir(img_dir):
    if not img_file.endswith('_test.jpg'):
        continue
        
    if not img_file in img2id:
        img_size = rate_map[img_file][2]
        
        r_image_id += 1
        img_id = r_image_id
        img2id[img_file] = img_id
        
        coco_images.append({
                    'id':r_image_id,
                    'width':img_size[0],
                    'height':img_size[1],
                    'file_name':img_file                                        
                })

In [14]:
import json
ann_dir = r'E:\users\minghaoli\ICDAR2013 Table COCO\data\ICDAR2013_table_coco\annotations'

COCO = {'images':coco_images, 'annotations':coco_annotations, 'categories':coco_categories}
json.dump(COCO, open(os.path.join(ann_dir, 'table_ICDAR2013_test.json'), 'w'))

In [16]:
coco_annotations

[{'id': 1,
  'image_id': 1,
  'category_id': 1,
  'segmentation': [],
  'area': 271222,
  'bbox': [277, 830, 1061, 255]},
 {'id': 2,
  'image_id': 1,
  'category_id': 1,
  'segmentation': [],
  'area': 507067,
  'bbox': [280, 1186, 1061, 477]},
 {'id': 3,
  'image_id': 1,
  'category_id': 1,
  'segmentation': [],
  'area': 323268,
  'bbox': [283, 1764, 1038, 311]},
 {'id': 4,
  'image_id': 2,
  'category_id': 1,
  'segmentation': [],
  'area': 773057,
  'bbox': [283, 263, 1050, 736]},
 {'id': 5,
  'image_id': 2,
  'category_id': 1,
  'segmentation': [],
  'area': 722276,
  'bbox': [280, 1100, 1061, 680]},
 {'id': 6,
  'image_id': 3,
  'category_id': 1,
  'segmentation': [],
  'area': 743908,
  'bbox': [286, 263, 1058, 702]},
 {'id': 7,
  'image_id': 3,
  'category_id': 1,
  'segmentation': [],
  'area': 294405,
  'bbox': [291, 1091, 1038, 283]},
 {'id': 8,
  'image_id': 4,
  'category_id': 1,
  'segmentation': [],
  'area': 386792,
  'bbox': [344, 588, 1063, 363]},
 {'id': 9,
  'image_